リメイク版動画＃１

In [2]:
#インポート文まとめ

from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.request import urlopen
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options
import traceback
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pickle
import scraping
import create_rawdf
from pathlib import Path

%load_ext autoreload

In [32]:
# 最初に実行
%autoreload

In [62]:
from urllib.request import urlopen
import pandas as pd

#url = "https://db.netkeiba.com/race/202304040509"
#html = urlopen(url).read()

#pd.read_html(html)[0]

2023年1月の開催日程取得　スクレイピング

In [24]:
from bs4 import BeautifulSoup
import re
#url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
#html = urlopen(url).read()

#soup = BeautifulSoup(html)
#a_list = soup.find("table",class_= "Calendar_Table").find_all("a")

#kaisai_date_list = []
#for a in a_list:
#    kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
#    kaisai_date_list.append(kaisai_date)




In [ ]:
#引数に該当するカレンダーページを取得する関数
def scrape_kaisai_date(year_,month_):
    url = f"https://race.netkeiba.com/top/calendar.html?year={year_}&month={month_}"
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
    kaisai_date_list = []
    for a in a_list:
        kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
        kaisai_date_list.append(kaisai_date)
    return kaisai_date_list

#実行例
#scrape_kaisai_date(year_=2023,month_=1)    

In [ ]:
#scrape_kaisai_date 年間の各月のレース日程を自動取得するように改変したい
def scrape_kaisai_date(from_,to_):
    url = f"https://race.netkeiba.com/top/calendar.html?year={year_}&month={month_}"
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
    kaisai_date_list = []
    for a in a_list:
        kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
        kaisai_date_list.append(kaisai_date)
    return kaisai_date_list


url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
html = urlopen(url).read()

2023年1月以外のレース日程も取得する

In [ ]:
#年月を取得する方法
#date_rangeのfreq属性について　Mは月ごとに区切り、Sは月始めを指定する設定
from_="2023-01"
to_="2023-12"

for date in pd.date_range(from_,to_,freq="MS"):
    year = date.year
    month = date.month
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

In [59]:
#scrape_kaisai_date 年間の各月のレース日程を自動取得するように上記の年月取得方法を組み込み
import time
from tqdm.notebook import tqdm

def scrape_kaisai_date(from_,to_):
    kaisai_date_list = []
    for date in tqdm(pd.date_range(from_,to_,freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        html = urlopen(url).read()
        time.sleep(1) #1秒の感覚をあけてスクレイピングするように絶対やる　→　Webページへのアクセス負荷とならないような配慮
        soup = BeautifulSoup(html)
        a_list = soup.find("table",class_= "Calendar_Table").find_all("a")
        
        for a in a_list:
            kaisai_date =re.findall(r"kaisai_date=(\d{8})",a["href"])[0]
            kaisai_date_list.append(kaisai_date)
        
    return kaisai_date_list

In [60]:
scrape_kaisai_date(from_="2023-01",to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129',
 '20230204',
 '20230205',
 '20230211',
 '20230212',
 '20230218',
 '20230219',
 '20230225',
 '20230226',
 '20230304',
 '20230305',
 '20230311',
 '20230312',
 '20230318',
 '20230319',
 '20230325',
 '20230326',
 '20230401',
 '20230402',
 '20230408',
 '20230409',
 '20230415',
 '20230416',
 '20230422',
 '20230423',
 '20230429',
 '20230430',
 '20230506',
 '20230507',
 '20230513',
 '20230514',
 '20230520',
 '20230521',
 '20230527',
 '20230528',
 '20230603',
 '20230604',
 '20230610',
 '20230611',
 '20230617',
 '20230618',
 '20230624',
 '20230625',
 '20230701',
 '20230702',
 '20230708',
 '20230709',
 '20230715',
 '20230716',
 '20230722',
 '20230723',
 '20230729',
 '20230730',
 '20230805',
 '20230806',
 '20230812',
 '20230813',
 '20230819',
 '20230820',
 '20230826',
 '20230827',
 '20230902',
 '20230903',
 '20230909',
 '20230910',
 '20230916',
 '20230917',
 '20230918',

ここまでの内容をscraping.pyに移行してます！！

リメイク版＃２
上記で取得した日程から今度はレースごとのURLを抽出する方法を考える

In [64]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"
html= urlopen(url).read()
soup = BeautifulSoup(html)

In [65]:
#soup.find("div",class_="RaceList_DataItem")

上記だと取得できないのでChromeDriverを使用する（javaScriptで画面を動的に制御しているため）
ChromeDriverのため以下パッケージをインストールする

In [66]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [67]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [70]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

driver_path = ChromeDriverManager().install()

'C:\\Users\\hisaf\\.wdm\\drivers\\chromedriver\\win64\\130.0.6723.69\\chromedriver-win32/chromedriver.exe'

In [74]:
driver = webdriver.Chrome(service=Service(driver_path))

In [79]:
driver.get(url)

In [84]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")

In [85]:
li = li_list[0]

In [93]:
href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
race_id = re.findall(r"race_id=(\d{12})", href)[0]

In [95]:
race_id_list = []
for li in li_list:
    href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    # print(race_id)
    race_id_list.append(race_id)

In [97]:
driver.quit()

前回のソースコードとの連結を考える
関数化する

In [3]:
import scraping

kaisai_date_list = scraping.scrape_kaisai_date(from_="2023-01",to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [1]:
from selenium.webdriver.chrome.options import Options
import traceback

def scrape_race_id_list(kaisai_date_list: list[str]):
    options = Options()
    options.add_argument("--headless")#バックグラウンドで実行
    driver_path = ChromeDriverManager().install()
    race_id_list = []

    with webdriver.Chrome(service=Service(driver_path),options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME,"a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc())
                break
    return race_id_list

In [6]:
race_id_list = scrape_race_id_list(kaisai_date_list)

  0%|          | 0/107 [00:00<?, ?it/s]

In [8]:
with open("race_id_list.pickle","wb") as f:
    pickle.dump(race_id_list,f)

リメイク版動画＃３

In [ ]:
url = "https://db.netkeiba.com/race/202306010101"
html = urlopen(url).read()
pd.read_html(html)[0]

In [117]:
import pickle

with open("race_id_list.pickle","rb") as f:
    race_id_list = pickle.load(f)

In [134]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    # print(url)

In [127]:
with open("data/html/202306010102.bin","wb") as f:
    f.write(html)

In [10]:
from pathlib import Path
HTML_DIR=Path("data","html")

In [165]:
HTML_DIR/"202306010101.bin"

WindowsPath('data/html/202306010101.bin')

In [166]:
with open(HTML_DIR/"202306010101.bin","wb") as f:
    f.write(html)

In [141]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    time.sleep(1)
    with open("data/html/"f"{race_id}.bin","wb") as f:
        f.write(html)


In [172]:
def scrape_html_race(race_id_list:list[str],save_dir:Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin"
        if filepath.is_file():
            print(f"skipped:{race_id}")
        else:
            url = f"https://db.netkeiba.com/race/{race_id}"
            html = urlopen(url).read()
            time.sleep(1)
            
            with open(filepath,"wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [174]:
html_path_list = scrape_html_race(race_id_list[:12],HTML_DIR)

  0%|          | 0/12 [00:00<?, ?it/s]

skipped:202306010101
skipped:202306010102
skipped:202306010103
skipped:202306010104
skipped:202306010105
skipped:202306010106
skipped:202306010107
skipped:202306010108
skipped:202306010109
skipped:202306010110


In [11]:
HTML_RACE_DIR = HTML_DIR / "race"

In [14]:
html_paths_race = list(HTML_RACE_DIR.glob("*"))

In [28]:
dfs = {}
for html_path in html_paths_race:
    with open(html_path, "rb") as f:
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        df.index = [race_id]*len(df)
        dfs[race_id] = df
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"

In [27]:
pd.concat(dfs.values())

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN,1.2,1.0,452(-4),[東] 鈴木伸尋
202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ,4.1,2.0,454(+2),[東] 杉浦宏昭
202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2,59.9,6.0,438(-6),[西] 羽月友彦
202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2,16.6,3.0,450(+2),[西] 武幸四郎
202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2,23.9,5.0,434(-10),[西] 今野貞一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4,62.8,11.0,394(+2),[西] 松永幹夫
202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2,125.6,12.0,470(-4),[西] 安田隆行
202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4,9.8,5.0,496(+8),[西] 森田直行
202310030812,11,4,4,ハイグッドエース,牡4,55.0,西塚洸二,2:43.7,3/4,37.2,10.0,440(-2),[西] 大根田裕


In [29]:
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_paths_race):
        with open(html_path, "rb") as f:
            race_id = html_path.stem
            html = f.read()
            df = pd.read_html(html)[0]
            df.index = [race_id]*len(df)
            dfs[race_id] = df
    concat_df = pd.concat(dfs.values())
    concat_df.index.name = "race_id"
    return concat_df

In [31]:
results = create_results(html_path_list= html_paths_race)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [32]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN,1.2,1.0,452(-4),[東] 鈴木伸尋
202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ,4.1,2.0,454(+2),[東] 杉浦宏昭
202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2,59.9,6.0,438(-6),[西] 羽月友彦
202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2,16.6,3.0,450(+2),[西] 武幸四郎
202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2,23.9,5.0,434(-10),[西] 今野貞一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4,62.8,11.0,394(+2),[西] 松永幹夫
202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2,125.6,12.0,470(-4),[西] 安田隆行
202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4,9.8,5.0,496(+8),[西] 森田直行


リメイク版動画＃４

In [33]:
with open("results.pickle","rb") as f:
    results = pickle.load(f)

In [35]:
url = "https://db.netkeiba.com/horse/2020103575"

In [36]:
html = urlopen(url).read()

In [ ]:
pd.read_html(html)[0]

In [11]:
url = "https://db.netkeiba.com/race/202306010101"
html = urlopen(url).read()
soup = BeautifulSoup(html,"lxml")

In [12]:
table = soup.find("table",class_="race_table_01 nk_tb_common")

In [13]:
#/horse/から始まるhref属性を持つaタグの一覧を取得する
a_list = table.find_all("a",href=re.compile(r"^/horse/"))

In [14]:
a = a_list[0]
a["href"]

'/horse/2020103575/'

In [76]:
re.findall(r"\d{10}",a["href"])[0]

'2020103575'

In [15]:
horse_id_list = []
for a in a_list:
    horse_id = re.findall(r"\d{10}",a["href"])[0]
    #print(horse_id)
    horse_id_list.append(horse_id)

In [16]:
horse_id_list

['2020103575',
 '2020107073',
 '2020102562',
 '2020106345',
 '2020100039',
 '2020104817',
 '2020104295',
 '2020105491',
 '2020104358',
 '2020105324',
 '2020103960',
 '2020102110',
 '2020102145',
 '2020101415',
 '2020106714',
 '2020100424']

In [6]:
HTML_RACE_DIR = Path("data","html","race")

In [17]:
html_paths_race = list(HTML_RACE_DIR.glob("*.bin"))
html_paths_race

[WindowsPath('data/html/race/202301010101.bin'),
 WindowsPath('data/html/race/202301010102.bin'),
 WindowsPath('data/html/race/202301010103.bin'),
 WindowsPath('data/html/race/202301010104.bin'),
 WindowsPath('data/html/race/202301010105.bin'),
 WindowsPath('data/html/race/202301010106.bin'),
 WindowsPath('data/html/race/202301010107.bin'),
 WindowsPath('data/html/race/202301010108.bin'),
 WindowsPath('data/html/race/202301010109.bin'),
 WindowsPath('data/html/race/202301010110.bin'),
 WindowsPath('data/html/race/202301010111.bin'),
 WindowsPath('data/html/race/202301010112.bin'),
 WindowsPath('data/html/race/202301010201.bin'),
 WindowsPath('data/html/race/202301010202.bin'),
 WindowsPath('data/html/race/202301010203.bin'),
 WindowsPath('data/html/race/202301010204.bin'),
 WindowsPath('data/html/race/202301010205.bin'),
 WindowsPath('data/html/race/202301010206.bin'),
 WindowsPath('data/html/race/202301010207.bin'),
 WindowsPath('data/html/race/202301010208.bin'),
 WindowsPath('data/h

In [33]:
results = create_rawdf.create_results(html_paths_race[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
results.isnull().sum()

着 順            0
枠 番            0
馬 番            0
馬名             0
性齢             0
斤量             0
騎手             0
タイム            0
着差            10
単勝             0
人 気            0
馬体重            0
調教師            0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
dtype: int64

In [26]:
results.columns = results.columns.str.replace(" ","")

In [27]:
results.columns = results.columns.str.replace(" ","")

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気',
       '馬体重', '調教師', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id'],
      dtype='object')

In [28]:
save_dir = Path("data","rawdf")
save_dir.mkdir()

In [31]:
results.to_csv(save_dir/"results.csv",sep="\t")

In [37]:
horse_id_list = results["horse_id"].unique()

In [38]:
len(horse_id_list)

118

In [43]:
html_paths_horse = scraping.scrape_html_horse(horse_id_list= horse_id_list)

  0%|          | 0/118 [00:00<?, ?it/s]

In [45]:
horse_results = create_rawdf.create_horse_results(html_path_list = html_paths_horse)

  0%|          | 0/118 [00:00<?, ?it/s]

In [46]:
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
horse_id,,,,,,,,,,,,,,,,,,,,,
2021101429,2024/10/06,4東京2,曇,8,3歳以上2勝クラス,NaN,13,6,8,1.9,...,-0.5,**,6-6,33.8-34.5,33.8,456(-2),NaN,NaN,(ニコラウス),1140.0
2021101429,2024/09/01,3新潟8,晴,9,飯豊特別(2勝クラス),NaN,7,7,7,3.0,...,0.4,**,4-4,35.2-34.4,34.4,458(-2),NaN,NaN,セントメモリーズ,625.2
2021101429,2024/08/10,3新潟1,晴,10,新発田城特別(2勝クラス),NaN,14,8,13,11.3,...,0.0,**,10-10,33.5-35.0,33.9,460(-4),NaN,NaN,サクセスアイ,629.2
2021101429,2024/04/06,3中山5,曇,11,ニュージーランドT(GII),NaN,16,8,16,75.3,...,0.9,**,14-16-16,36.0-35.5,34.9,464(0),NaN,NaN,エコロブルーム,NaN
2021101429,2024/03/16,1中京3,晴,11,中スポ賞ファルコンS(GIII),NaN,17,2,3,10.1,...,0.9,**,12-12,33.8-34.7,34.7,464(+8),NaN,NaN,ダノンマッキンリー,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018105226,2021/02/13,1東京5,晴,7,3歳1勝クラス,NaN,16,1,1,20.2,...,0.7,**,4-3,34.4-35.5,35.9,454(+8),NaN,NaN,レイモンドバローズ,NaN
2018105226,2020/11/22,5東京6,晴,3,2歳未勝利,NaN,18,7,14,9.7,...,0.0,**,2-2,35.6-34.6,34.4,446(0),NaN,NaN,(アップリバー),510.0
2018105226,2020/11/01,3福島2,晴,2,2歳未勝利,NaN,15,1,1,1.8,...,0.1,**,4-3,34.0-35.8,35.6,446(-2),NaN,NaN,テセウス,130.0


リメイク版動画＃6

In [3]:
DATA_DIR = Path("data")
HTML_DIR = DATA_DIR / "html"
HTML_RACE_DIR = HTML_DIR / "race"

In [4]:
html_path_list = list(HTML_RACE_DIR.glob("*.bin"))


[WindowsPath('data/html/race/202301010101.bin'),
 WindowsPath('data/html/race/202301010102.bin'),
 WindowsPath('data/html/race/202301010103.bin'),
 WindowsPath('data/html/race/202301010104.bin'),
 WindowsPath('data/html/race/202301010105.bin'),
 WindowsPath('data/html/race/202301010106.bin'),
 WindowsPath('data/html/race/202301010107.bin'),
 WindowsPath('data/html/race/202301010108.bin'),
 WindowsPath('data/html/race/202301010109.bin'),
 WindowsPath('data/html/race/202301010110.bin')]

In [5]:
with open(html_path_list[0],"rb") as f:
    html = f.read()

In [6]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>2\xba\xd0\xcc\xa4\xbe\xa1\xcd\xf8\xa1\xc32023\xc7\xaf7\xb7\xee22\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [7]:
soup = BeautifulSoup(html,"lxml")

In [8]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>2歳未勝利｜2023年7月22日 | 競馬データベース - netkeiba</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎

In [10]:
soup_info = soup.find("div",class_="data_intro")

In [11]:
soup_info.find("h1").text

'2歳未勝利'

In [19]:
tmp = soup_info.find("p").text.replace(" ","")
tmp

'\n\n芝右1200m\xa0/\xa0天候:晴\xa0/\xa0芝:良\xa0/\xa0発走:09:50\n\n\n'

In [20]:
re.findall(r"[\w:]+",tmp)

['芝右1200m', '天候:晴', '芝:良', '発走:09:50']

In [25]:
tmp = soup_info.find_all("p")[1].text

In [26]:
re.findall(r"\w+",tmp)

['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']

In [27]:
info_dict = {}
info_dict["title"] = soup_info.find("h1").text
info_dict["info1"] = re.findall(
    r"[\w:]+",soup_info.find("p").text.replace(" ","")
)
info_dict["info2"] = re.findall(r"\w+",soup_info.find_all("p")[1].text)
info_dict

{'title': '2歳未勝利',
 'info1': ['芝右1200m', '天候:晴', '芝:良', '発走:09:50'],
 'info2': ['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']}

In [31]:
pd.DataFrame().from_dict(info_dict,orient="index").T

,title,info1,info2
0,2歳未勝利,"[芝右1200m, 天候:晴, 芝:良, 発走:09:50]","[2023年7月22日, 1回札幌1日目, 2歳未勝利, 混, 指, 馬齢]"


In [34]:
race_info = create_rawdf.create_race_info(html_path_list)

AttributeError: module 'create_rawdf' has no attribute 'create_race_info'